# モデルをトレーニングする

機械学習の中心的な目的は、アプリケーションで使用できる予測モデルをトレーニングすることです。Azure Machine Learning では、スクリプトを使用して、Scikit-Learn、TensorFlow、PyTorch、SparkML などの一般的な機械学習フレームワークを活用してモデルをトレーニングできます。これらのトレーニング スクリプトを実験として実行し、メトリックと出力 (トレーニング済みのモデルを含む) を追跡できます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## トレーニング スクリプトを作成する

Python スクリプトを使用して、糖尿病データに基づいて機械学習モデルをトレーニングします。スクリプトとデータ ファイル用のフォルダーを作成することから始めましょう。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

これで、トレーニング スクリプトを作成してフォルダーに保存する準備が整いました。

> **注**: このコードはスクリプトを*作成*しますが実行しません!

In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 正規化ハイパーパラメーターを設定する
reg = 0.01

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# トレーニング済モデルを出力フォルダーに保存する。
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## トレーニング スクリプトを実験として実行する

これで、スクリプトを実験として実行する準備が整いました。既定の環境には **scikit-learn** パッケージが含まれていないため、明示的にこれを構成に追加する必要がある点に留意してください。Conda 環境は、初めて 実験を実行するとオンデマンドで構築され、同じ構成を使用する将来の実行用にキャッシュされます。このため、初回の実行には少し時間がかかります。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# 実験用 Python 環境を作成する
sklearn_env = Environment("sklearn-env")

# 必要なパッケージがインストールされていることを確認する (既定で scikit-learn と Azure ML が必要)
packages = CondaDependencies.create(pip_packages=['scikit-learn','azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                environment=sklearn_env) 

# 実験実行を送信する
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# ノートブック ウィジェットで実行中の実験実行を表示する
RunDetails(run).show()

# 実験実行が完了するまでブロックする
run.wait_for_completion()

**Run** オブジェクトからメトリックと出力を取得できます。

In [ ]:
# メトリックの記録とファイルを取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## トレーニングされたモデルを登録する

実験の出力には、トレーニング済みモデル ファイル (**diabetes_model.pkl**) が含まれることに注意してください。このモデルを Azure Machine Learning ワークスペースに登録すると、モデルのバージョンを追跡し、後で取得できるようになります。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## パラメーター化されたトレーニング スクリプトを作成する

スクリプトにパラメーターを追加することで、トレーニング実験の柔軟性を高めることができ、同じトレーニング実験を異なる設定で繰り返すことができます。この場合、モデルのトレーニング時にロジスティック回帰アルゴリズムで使用される正規化率のパラメーターを追加します。

パラメーター化されたスクリプトとトレーニング データ用のフォルダーをもう一度作成します。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

次に、正規化率ハイパーパラメーターの引数でスクリプトを作成します。引数は、Python **argparse.ArgumentParser** オブジェクトを使用して読み取られます。

In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 実験実行コンテキストを取得する
run = Run.get_context()

# 正規化ハイパーパラメーターを設定する
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# 糖尿病データセットを読み込む
print("Loading Data...")
# 糖尿病データセットを読み込む
diabetes = pd.read_csv('diabetes.csv')

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 引数でスクリプトを実行する

以前と同様にスクリプトを実験として実行し、作成した環境を再使用します。ただし、ここではスクリプトが引数として想定している **--reg_rate** パラメーターを提供する必要があります。

In [ ]:
# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=sklearn_env) 

# 実験を送信する
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

もう一度、完了した実行からメトリックと出力を取得できます。

In [ ]:
# メトリックの記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## モデルの新しいバージョンを登録する

新しいモデルをトレーニングしたので、ワークスペースに新しいバージョンとして登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

ワークスペースの登録済みモデルは、[Azure Machine Learning studio](https://ml.azure.com) の「**モデル**」ページでも表示できます。

探索が終了したら、このノートブックを閉じて、コンピューティング インスタンスをシャットダウンできます。